<a href="https://colab.research.google.com/github/kshyamano/hello_world/blob/master/Minor_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets accelerate bitsandbytes peft

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

# 1. Load dataset (JSON with 'question' and 'answer'). Replace path as needed.
data_files = {"train": "qa_data.json"}  # Path to your JSON dataset file
raw_datasets = load_dataset("json", data_files=data_files)
train_dataset = raw_datasets["train"]

# 2. Initialize tokenizer and model (use 4-bit quantization for memory efficiency on free GPU).
model_name = "deepseek-ai/deepseek-llm-7b-base"  # or a smaller DeepSeek model if available
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# 3. Configure LoRA: rank r=8 (~1% of params for a 7B model), alpha=16, dropout=0.1.
# Task is causal LM, inference_mode=False enables training.
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)
model = get_peft_model(model, lora_config)  # freeze base model, only LoRA adapters are trainable



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

KeyError: 'question'

In [4]:
# 1. Load dataset (JSON with 'question' and 'answer'). Replace path as needed.
data_files = {"train": "qa_data.json"}  # Path to your JSON dataset file
raw_datasets = load_dataset("json", data_files=data_files)
train_dataset = raw_datasets["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# 4. Preprocess: combine question and answer, mask question tokens in labels.
def preprocess_fn(examples):
    inputs, masks, labels = [], [], []
    for q, a in zip(examples["question"], examples["answer"]):
        prompt = f"Question: {q}\\nAnswer: {a}"
        tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
        input_ids = tokenized["input_ids"]
        # Compute token length of "Question: ... Answer:" to mask question part
        q_tokens = tokenizer(f"Question: {q}\\nAnswer:", add_special_tokens=False)
        q_len = len(q_tokens["input_ids"])
        label_ids = input_ids.copy()
        label_ids[:q_len] = [-100] * q_len  # mask question tokens (do not train on them)
        inputs.append(input_ids)
        masks.append(tokenized["attention_mask"])
        labels.append(label_ids)
    return {"input_ids": inputs, "attention_mask": masks, "labels": labels}

tokenized_dataset = train_dataset.map(
    preprocess_fn,
    batched=True,
    remove_columns=train_dataset.column_names
)

# 5. Setup Trainer with a causal LM data collator.
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="deepseek_lora_output",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # small batch for memory constraints
    logging_steps=10,
    save_strategy="epoch"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# 6. Fine-tune the model (only LoRA adapter weights will be updated).
trainer.train()

# 7. Save the LoRA adapter (this saves only the adapter weights and config).
model.save_pretrained("deepseek_lora_adapter")


<ipython-input-6-2e7f7f201a69>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kshyamano (kshyamano-iit-delhi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [3]:
print(train_dataset.column_names)


['churn_reason', 'recommended_policy']


In [1]:
def ask(question):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()

# Try a question
response = ask("What is machine learning?")
print(response)


NameError: name 'tokenizer' is not defined

In [16]:
response = ask(""" Given the telecom data analysis for Chunk 0, what are the reasons for churn and what policy should be applied to reduce the churn ratenum__tenure                            0.921710
cat__Contract_Month-to-month           0.897793
cat__OnlineSecurity_No                 0.489412
num__MonthlyCharges                    0.372965
num__TotalCharges                      0.316216
cat__PaymentMethod_Electronic check    0.304826
cat__InternetService_Fiber optic       0.293343
cat__gender_Female                     0.281654
cat__TechSupport_No                    0.268071
cat__PaperlessBilling_Yes              0.257186
dtype: float32

🎯 Target value distribution for Chunk 50:

target
0         41
1         41
               """)
print(response)


The telecom data analysis for Chunk 50, what are the reasons for churn and what policy should be applied to reduce the churn rate?

The target value distribution for Chunk 50 is:

0         41
1         41
               
The target value distribution for Chunk 50 shows that there is a 50/50 chance of churning.

The reasons for churning in this case are that the customers have been


In [12]:
print(response)

It's a very good question and I'm glad you asked.
There are some very basic things that you need to know about your policy.
First and foremost, it's important to know what is covered and what is not covered. If you have a car, you need to know what is covered and what isn't covered.
If you have a house, you need to know what is covered and what isn't covered. If you have a pet, you need
